In [110]:
from functools import partial
# JPEG stuff
def jpeg_compress(x, quality=75):
    return tf.image.decode_jpeg(
        tf.image.encode_jpeg(
            x, format='grayscale', quality=quality),
        channels=1)

In [111]:
"""
This tutorial shows how to generate adversarial examples using FGSM
and train a model using adversarial training with TensorFlow.
It is very similar to mnist_tutorial_keras_tf.py, which does the same
thing but with a dependence on keras.
The original paper can be found at:
https://arxiv.org/abs/1412.6572
"""
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function
from __future__ import unicode_literals

import logging
import numpy as np
import tensorflow as tf

from cleverhans.loss import CrossEntropy
from cleverhans.dataset import MNIST
from cleverhans.utils_tf import model_eval
from cleverhans.train import train
from cleverhans.attacks import FastGradientMethod
from cleverhans.utils import AccuracyReport, set_log_level
from cleverhans_tutorials.tutorial_models import ModelBasicCNN

NB_EPOCHS = 6
BATCH_SIZE = 128
LEARNING_RATE = 0.001
CLEAN_TRAIN = True
BACKPROP_THROUGH_ATTACK = False
NB_FILTERS = 64


def mnist_tutorial(train_start=0, train_end=60000, test_start=0,
                   test_end=10000, nb_epochs=NB_EPOCHS, batch_size=BATCH_SIZE,
                   learning_rate=LEARNING_RATE,
                   clean_train=CLEAN_TRAIN,
                   testing=False,
                   backprop_through_attack=BACKPROP_THROUGH_ATTACK,
                   nb_filters=NB_FILTERS, num_threads=None,
                   label_smoothing=0.1):
  """
  MNIST cleverhans tutorial
  :param train_start: index of first training set example
  :param train_end: index of last training set example
  :param test_start: index of first test set example
  :param test_end: index of last test set example
  :param nb_epochs: number of epochs to train model
  :param batch_size: size of training batches
  :param learning_rate: learning rate for training
  :param clean_train: perform normal training on clean examples only
                      before performing adversarial training.
  :param testing: if true, complete an AccuracyReport for unit tests
                  to verify that performance is adequate
  :param backprop_through_attack: If True, backprop through adversarial
                                  example construction process during
                                  adversarial training.
  :param label_smoothing: float, amount of label smoothing for cross entropy
  :return: an AccuracyReport object
  """

  # Object used to keep track of (and return) key accuracies
  report = AccuracyReport()

  # Set TF random seed to improve reproducibility
  tf.set_random_seed(1234)

  # Set logging level to see debug information
  set_log_level(logging.DEBUG)

  # Create TF session
  if num_threads:
    config_args = dict(intra_op_parallelism_threads=1)
  else:
    config_args = {}
  sess = tf.Session(config=tf.ConfigProto(**config_args))

  # Get MNIST data
  mnist = MNIST(train_start=train_start, train_end=train_end,
                test_start=test_start, test_end=test_end)
  x_train, y_train = mnist.get_set('train')
  x_test, y_test = mnist.get_set('test')

  # Use Image Parameters
  img_rows, img_cols, nchannels = x_train.shape[1:4]
  nb_classes = y_train.shape[1]

  # Define input TF placeholder
  x = tf.placeholder(tf.float32, shape=(None, img_rows, img_cols,
                                        nchannels))
  y = tf.placeholder(tf.float32, shape=(None, nb_classes))

  # Train an MNIST model
  train_params = {
      'nb_epochs': nb_epochs,
      'batch_size': batch_size,
      'learning_rate': learning_rate
  }
  eval_params = {'batch_size': batch_size}
  fgsm_params = {
      'eps': 0.3,
      'clip_min': 0.,
      'clip_max': 1.
  }
  rng = np.random.RandomState([2017, 8, 30])

  def do_eval(preds, x_set, y_set, report_key, report_text=None):
    acc = model_eval(sess, x, y, preds, x_set, y_set, args=eval_params)
    setattr(report, report_key, acc)
    if report_text:
      print('Test accuracy on %s examples: %0.4f' % (report_text, acc))

  if clean_train:
    model = ModelBasicCNN('model1', nb_classes, nb_filters)
    preds = model.get_logits(x)
    loss = CrossEntropy(model, smoothing=label_smoothing)

    def evaluate():
      do_eval(preds, x_test, y_test, 'clean_train_clean_eval', 'legitimate')

    train(sess, loss, x_train, y_train, evaluate=evaluate,
          args=train_params, rng=rng, var_list=model.get_params())

    # Calculate training error
    if testing:
      do_eval(preds, x_train, y_train, 'train_clean_train_clean_eval', None)

    # Initialize the Fast Gradient Sign Method (FGSM) attack object and
    # graph
    fgsm = FastGradientMethod(model, sess=sess)
    adv_x = fgsm.generate(x, **fgsm_params)
    preds_adv = model.get_logits(adv_x)

    # Evaluate the accuracy of the MNIST model on adversarial examples
    do_eval(preds_adv, x_test, y_test, 'clean_train_adv_eval', 'adversarial')

    # Calculate training error
    if testing:
      do_eval(preds_adv, x_train, y_train, 'train_clean_train_adv_eval', None)
    
    for quality in range(10, 100, 5):
        fn = partial(jpeg_compress, quality=quality)
        jpeg_adv_x = tf.map_fn(fn, preprocessing_fn(adv_x))
        # TODO: Change this bullshit to use https://www.tensorflow.org/api_docs/python/tf/image/convert_image_dtype
        # TODO: Refactor
        foo = tf.cast(jpeg_adv_x / 255, tf.float32)
        foo.set_shape((None, 28, 28, 1))
        preds_jpeg_adv = model.get_logits(foo)
        do_eval(preds_jpeg_adv, x_test, y_test, 'clean_train_adv_eval', 'jpeg (quality = %d)' % quality)

    print('Repeating the process, using adversarial training')

  # Create a new model and train it to be robust to FastGradientMethod
  model2 = ModelBasicCNN('model2', nb_classes, nb_filters)
  fgsm2 = FastGradientMethod(model2, sess=sess)

  def attack(x):
    return fgsm2.generate(x, **fgsm_params)

  loss2 = CrossEntropy(model2, smoothing=label_smoothing, attack=attack)
  preds2 = model2.get_logits(x)
  adv_x2 = attack(x)

  if not backprop_through_attack:
    # For the fgsm attack used in this tutorial, the attack has zero
    # gradient so enabling this flag does not change the gradient.
    # For some other attacks, enabling this flag increases the cost of
    # training, but gives the defender the ability to anticipate how
    # the atacker will change their strategy in response to updates to
    # the defender's parameters.
    adv_x2 = tf.stop_gradient(adv_x2)
  preds2_adv = model2.get_logits(adv_x2)

  def evaluate2():
    # Accuracy of adversarially trained model on legitimate test inputs
    do_eval(preds2, x_test, y_test, 'adv_train_clean_eval', False)
    # Accuracy of the adversarially trained model on adversarial examples
    # Replace this where you perturb the JPEG images before training I guess?
    do_eval(preds2_adv, x_test, y_test, 'adv_train_adv_eval', True)

  # Perform and evaluate adversarial training
  train(sess, loss2, x_train, y_train, evaluate=evaluate2,
        args=train_params, rng=rng, var_list=model2.get_params())

  # Calculate training errors
  if testing:
    do_eval(preds2, x_train, y_train, 'train_adv_train_clean_eval')
    do_eval(preds2_adv, x_train, y_train, 'train_adv_train_adv_eval')

  return report


def main(argv=None):
  from cleverhans_tutorials import check_installation
  check_installation(__file__)

  mnist_tutorial(nb_epochs=NB_EPOCHS, batch_size=BATCH_SIZE,
                 learning_rate=LEARNING_RATE,
                 clean_train=CLEAN_TRAIN,
                 backprop_through_attack=BACKPROP_THROUGH_ATTACK,
                 nb_filters=NB_FILTERS)

In [3]:
3

3

In [18]:
model, x, adv_x, sess, fgsm_params = mnist_tutorial(nb_epochs=1)

/opt/conda/lib/python3.6/site-packages/cleverhans/utils_tf.py:511: UserWarning: No GPUS, running on CPU
  warnings.warn("No GPUS, running on CPU")


num_devices:  1


[INFO 2019-02-06 21:33:35,098 cleverhans] Epoch 0 took 121.65795183181763 seconds


Test accuracy on legitimate examples: 0.9876
Test accuracy on adversarial examples: 0.0685


In [109]:
tf.reset_default_graph()

In [19]:
x

<tf.Tensor 'Placeholder:0' shape=(?, 28, 28, 1) dtype=float32>

In [20]:
adv_x

<tf.Tensor 'Identity_1:0' shape=(?, 28, 28, 1) dtype=float32>

In [26]:
preprocessing_fn = \
    (lambda x: tf.cast(
        255. * (x + 1.) / 2.,
        tf.uint8)) \
    if not False else lambda x: x

In [85]:
post_fn = \
    (lambda x: tf.cast(
        x * (1 / 255.),
        tf.float32)) \
    if not False else lambda x: x

In [65]:
def postprocessing_fn(x):
    return tf.cast(x / 255., tf.float32)

In [67]:
adv_x

<tf.Tensor 'Identity_1:0' shape=(?, 28, 28, 1) dtype=float32>

In [66]:
preprocessing_fn(adv_x)

<tf.Tensor 'Cast_16:0' shape=(?, 28, 28, 1) dtype=uint8>

In [86]:
post_fn(preprocessing_fn(adv_x))

TypeError: unsupported operand type(s) for *: 'Tensor' and 'float'

In [39]:
#jpeg_adv_x = jpeg_compress(preprocessing_fn(adv_x))
#preprocessing_fn(adv_x).set_shape((None, None, None, 3))
rgb_adv_x = tf.image.grayscale_to_rgb(adv_x)
foo = preprocessing_fn(rgb_adv_x)
foo

<tf.Tensor 'Cast_10:0' shape=(?, 28, 28, 3) dtype=uint8>

In [55]:
tf.map_fn(jpeg_compress, preprocessing_fn(adv_x))

<tf.Tensor 'map_3/TensorArrayStack/TensorArrayGatherV3:0' shape=(?, ?, ?, 1) dtype=uint8>

In [45]:
foo.shape[1:4]

TensorShape([Dimension(28), Dimension(28), Dimension(3)])

In [53]:
preprocessing_fn(adv_x)

<tf.Tensor 'Cast_13:0' shape=(?, 28, 28, 1) dtype=uint8>

In [112]:
mnist_tutorial(nb_epochs=1)

/opt/conda/lib/python3.6/site-packages/cleverhans/utils_tf.py:511: UserWarning: No GPUS, running on CPU
  warnings.warn("No GPUS, running on CPU")


num_devices:  1


[INFO 2019-02-06 23:14:43,303 cleverhans] Epoch 0 took 296.7815635204315 seconds


Test accuracy on legitimate examples: 0.9876
Test accuracy on adversarial examples: 0.0685
Test accuracy on jpeg (quality = 10) examples: 0.1771
Test accuracy on jpeg (quality = 15) examples: 0.1747
Test accuracy on jpeg (quality = 20) examples: 0.1739
Test accuracy on jpeg (quality = 25) examples: 0.1726
Test accuracy on jpeg (quality = 30) examples: 0.1725
Test accuracy on jpeg (quality = 35) examples: 0.1721
Test accuracy on jpeg (quality = 40) examples: 0.1691
Test accuracy on jpeg (quality = 45) examples: 0.1689
Test accuracy on jpeg (quality = 50) examples: 0.1664
Test accuracy on jpeg (quality = 55) examples: 0.1669
Test accuracy on jpeg (quality = 60) examples: 0.1647
Test accuracy on jpeg (quality = 65) examples: 0.1640
Test accuracy on jpeg (quality = 70) examples: 0.1621
Test accuracy on jpeg (quality = 75) examples: 0.1628
Test accuracy on jpeg (quality = 80) examples: 0.1604
Test accuracy on jpeg (quality = 85) examples: 0.1616
Test accuracy on jpeg (quality = 90) examples

/opt/conda/lib/python3.6/site-packages/cleverhans/loss.py:41: UserWarning: callable attacks are deprecated, switch to an Attack subclass. callable attacks will not be supported after 2019-05-05.
  warnings.warn("callable attacks are deprecated, switch to an Attack "
/opt/conda/lib/python3.6/site-packages/cleverhans/utils_tf.py:511: UserWarning: No GPUS, running on CPU
  warnings.warn("No GPUS, running on CPU")


num_devices:  1


[INFO 2019-02-06 23:33:44,121 cleverhans] Epoch 0 took 462.59301805496216 seconds


Test accuracy on True examples: 0.8153


In [89]:
final = tf.cast(preprocessing_fn(adv_x) / 255, tf.float32)

In [91]:
model.get_logits(final)

ValueError: Tensor("model1/conv2d/kernel:0", shape=(8, 8, 1, 64), dtype=float32_ref) must be from the same graph as Tensor("truediv_40:0", shape=(?, 28, 28, 1), dtype=float32).

In [95]:
tf.float32_ref

AttributeError: module 'tensorflow' has no attribute 'float32_ref'

In [97]:
model.get_logits(adv_x)

ValueError: Tensor("model1/conv2d/kernel:0", shape=(8, 8, 1, 64), dtype=float32_ref) must be from the same graph as Tensor("Identity_1:0", shape=(?, 28, 28, 1), dtype=float32).

In [98]:
jpeg_adv_x = tf.map_fn(jpeg_compress, preprocessing_fn(adv_x))

In [99]:
jpeg_adv_x

<tf.Tensor 'map_4/TensorArrayStack/TensorArrayGatherV3:0' shape=(?, ?, ?, 1) dtype=uint8>

In [102]:
foo = tf.cast(jpeg_adv_x / 255, tf.float32)
foo.set_shape((None, 28, 28, 1))
foo

<tf.Tensor 'truediv_44:0' shape=(?, 28, 28, 1) dtype=float32>

In [103]:
model.get_logits(foo)

ValueError: Tensor("model1/conv2d/kernel:0", shape=(8, 8, 1, 64), dtype=float32_ref) must be from the same graph as Tensor("truediv_44:0", shape=(?, 28, 28, 1), dtype=float32).

In [108]:
list(range(10, 100, 5))

[10, 15, 20, 25, 30, 35, 40, 45, 50, 55, 60, 65, 70, 75, 80, 85, 90, 95]